In [1]:
reset()
%display latex
Partitions.options.latex="list"

# Demo: toric DT and PT theory

For toric $3$-folds $X$, every form of Donaldson--Thomas theory breaks down into contributions from vertices and edges of the toric skeleton of $X$. Edge contributions are the same across all theories, but vertices are different.

This file demonstrates how to work with these vertices and edges.

### 3d partitions with legs

In [2]:
load("partition_3d.sage")

The DT vertex, via localization, is a combinatorial sum over *3d partitions with legs*. These are implemented as `Partitions3d(lamb, mu, nu)` where $\lambda$, $\mu$, $\nu$ are integer partitions specifying the profiles of the legs.

Ordinary integer partitions are "2d" and have no infinite legs.

In [3]:
PPs = Partitions3d([2], [1], [3,1])
PPs

3d partitions with legs [2], [1], [3, 1]

Once legs are fixed, there is always a *minimal* 3d partition that has no extra boxes aside from those in the legs.

In [4]:
PPs.minimal_element()

3d partition with legs [2], [1], [3, 1] and extra boxes []

In [5]:
PPs.minimal_element().plot(colors=('green', 'yellow', 'white'))

Graphics3d Object

The command `PPs.with_num_boxes(n)` returns an iterator through all configurations with $n$ extra boxes on top of the minimal configuration.

In [6]:
PPs2 = list(PPs.with_num_boxes(2))
PPs2[1].plot(colors=('green', 'yellow', 'white'))

Graphics3d Object

The command `PPs.random_elemnt_with_num_boxes(n)` returns a random element with $n$ extra boxes. It is obtained iteratively by choosing, $n$ times in a row, a random permissible place to add a new box. Each choice is uniform but the overall result is not uniformly distributed.

In [7]:
PPs.random_element_with_num_boxes(25).plot(colors=('green', 'yellow', 'white'))

Graphics3d Object

Of course, when there are no legs, the result is just ordinary 3d partitions (also known as *plane partitions*). Their generating function is given by the well-known MacMahon formula.

$$ M(q) = \prod_{n > 0} \frac{1}{(1 - q^n)^n}. $$

In [8]:
R.<q> = LaurentSeriesRing(ZZ)

PPs = Partitions3d([], [], [])
sum( q^k * len(list(PPs.with_num_boxes(k))) for k in range(10) ).add_bigoh(10)

1 + q + 3*q^2 + 6*q^3 + 13*q^4 + 24*q^5 + 48*q^6 + 86*q^7 + 160*q^8 + 282*q^9 + O(q^10)

#### Extended example: topological vertex

The *topological vertex* $C_{\lambda,\mu,\nu}$ is, up to some prefactors, the generating function for 3d partitions with legs $\lambda$, $\mu$, $\nu$. It has an explicit formula, first given in [Okounkov-Reshetikhin-Vafa], in terms of skew Schur functions. We can verify this formula computationally.

The following function computes the topological vertex by counting (see formula 3.23 in [ORV]). Since there are some prefactors which involve $q^{1/2}$, we make the substitution $q \mapsto q^2$.

In [9]:
def compute_topological_vertex_by_counting(lamb, mu, nu, q, n=8):
    lamb, mu, nu = Partition(lamb), Partition(mu), Partition(nu) # in case

    PPs = Partitions3d(lamb, mu, nu)
    P = sum(q^(2*PP.volume()) for PP in PPs.up_to_num_boxes(n-1))
    P = P.truncate_laurentseries(P.valuation() + 2*n)

    partition_norm = lambda mu: sum(l^2 for l in mu)
    return P * prod((1 - q^(2*i))^i for i in range(1, n)) * \
         q^(partition_norm(lamb.conjugate()) + partition_norm(mu.conjugate()) + partition_norm(nu.conjugate()))

In [10]:
compute_topological_vertex_by_counting([3,2], [1], [2], q)

q^-2 + 3 + 8*q^2 + 17*q^4 + 34*q^6 + 62*q^8 + 109*q^10 + 181*q^12 + O(q^14)

The following function implements the skew Schur function $s_{\lambda/\mu}(q^{-\nu-\rho})$ where $\rho = (-1/2, -3/2, -5/2, \ldots)$. The result is returned as a series in $q$ with $n$ terms of precision (default $n=8$).

In [11]:
def skew_schur_q(lamb, mu, nu, q, n=8):
    if not lamb.contains(mu):
        return q.parent().zero()
    
    s = SymmetricFunctions(ZZ).s()
    sp = SkewPartition([lamb, mu])
    get = lambda mu, i: mu[i] if i < len(mu) else 0

    nvars = max(get(nu, 0), 1) * sum(nu) + n+2
    f = s.skew_schur(sp).expand(nvars, 'x')
    args = (q^(-get(nu, i) + i + 1/2) for i in range(nvars))
    fq = q.parent()(f(*args))

    return fq.truncate_laurentseries(fq.valuation() + 2*n)

Finally, this is the [Okounkov-Reshtikhin-Vafa] formula 3.15 for the topological vertex.

In [12]:
def compute_topological_vertex_by_formula(lamb, mu, nu, q, n=8):
    lamb, mu, nu = Partition(lamb), Partition(mu), Partition(nu) # in case

    partition_kappa = lambda mu: 2 * sum(j - i for i, j in mu.cells())
    prefactor = q^(-partition_kappa(lamb) - partition_kappa(nu)) * \
                skew_schur_q(nu.conjugate(), [], [], q^2, n)
    res = sum( ( skew_schur_q(lamb.conjugate(), eta, nu, q^2, n) * \
                 skew_schur_q(mu, eta, nu.conjugate(), q^2, n) 
                 for k in range(lamb.size()+1) for eta in Partitions(k, outer=lamb.conjugate()) ), q.parent().zero() )
    return prefactor * res.truncate_laurentseries(res.valuation() + 2*n)

Test that the formula matches our counting, for $10$ randomly-selected partitions $\lambda$, $\mu$, $\nu$.

In [13]:
def random_partition(size_up_to=8):
    from random import randint
    return Partitions(randint(0, size_up_to)).random_element()

lambs, mus, nus = [[random_partition(3) for _ in range(10)] for _ in range(3)]
all( compute_topological_vertex_by_formula(lamb, mu, nu, q) == 
     compute_topological_vertex_by_counting(lamb, mu, nu, q) for lamb, mu, nu in zip(lambs, mus, nus) )

True

### DT vertex in cohomology

In [14]:
load("bare_vertex.sage")

Let $\Pi(\lambda,\mu,\nu)$ denote the set of 3d partitions with legs $\lambda, \mu, \nu$. The DT vertex is

$$ V(\lambda, \mu, \nu; q) = \sum_{\pi \in \Pi(\lambda,\mu,\nu)} \frac{(-q)^{|\pi|}}{e(T_\pi \mathrm{Hilb}(\mathbb{C}^3))} \in H_{\mathsf{T}}^*(\mathrm{pt})_{\mathrm{loc}}((q)). $$

Here $\mathsf{T} = (\mathbb{C}^\times)^3$ acts on $\mathbb{C}^3$ by scaling; we denote the weights by $x$, $y$, and $z$. The denominator is the localization contribution of $\pi$; it has some explicit formula given in [MNOP1]. The variable $q$ is the "box-counting" parameter.

In [15]:
wR.<x,y,z> = LaurentPolynomialRing(QQ)
qR.<q> = PowerSeriesRing(wR.fraction_field())

#### Basics

The command `BareVertex(lamb, mu, nu, Cohomology)` creates the cohomological vertex with legs $\lambda$, $\mu$, $\nu$.

In [16]:
V000 = BareVertex([], [], [], Cohomology)
V000

Bare vertex with legs ([], [], []) using cohomology

The command `V.series_unreduced(n, x, y, z, q)` computes the first $n$ terms, in $q$, of the vertex $V$.

In [17]:
V000series = V000.series_unreduced(3, x, y, z, q)
pretty(V000series)

1 + (-(x + y)*(x + z)*(y + z)/(x*y*z))*q + (1/2*(x^2*y + x*y^2 + x^2*z - 3*x*y*z + y^2*z + x*z^2 + y*z^2)*(x + y)*(x + z)*(y + z)/(x^2*y^2*z^2))*q^2 + O(q^3)

The object `Cohomology`, in `setup.sage`, is a class containing some useful cohomological methods, in particular the method `Cohomology.measure(f)` which returns the Euler class $e(f)$. For instance, $T_\pi$ when $\pi$ is a single box is just $x + y + z - xy - yz - xz$, and this gives the linear term above.

In [18]:
-V000series.coefficients()[1] == Cohomology.measure(x + y + z - x*y - y*z - x*z)

True

#### Calabi-Yau limit

In the specialization $x + y + z = 0$, we have $e(T_\pi) = 0$ by self-duality. This is called the *Calabi-Yau limit*. Hence the vertex becomes the ordinary generating function of 3d partitions (with legs), also known as the topological vertex.

In [19]:
V000series(x=x, y=y, z=-x-y)

1 + q + 3*q^2 + O(q^3)

#### Reduced vs. unreduced series

Usually one wants to work with the *reduced* series $V'(\lambda, \mu, \nu) = V(\lambda,\mu,\nu) / V(\emptyset, \emptyset, \emptyset)$. This is implemented as `V.series(n, x, y, z, q)`.

In [20]:
V = BareVertex([1,1], [1], [3], Cohomology)
pretty( V.series(2, x, y, z, q) )

(1/2*(2*x - y)*(x - y + z)*(x - y)*(y - z)*y/((3*x + z)*(2*x + z)*(x + 2*z)*(x + z)^2))*q^-7 + (1/2*(9*x^3*y^2 - 9*x*y^4 - 6*x^4*z + 19*x^3*y*z + x^2*y^2*z - 10*x*y^3*z - 4*y^4*z - 10*x^3*z^2 + 15*x^2*y*z^2 - 3*x*y^2*z^2 - 4*y^3*z^2 - 4*x^2*z^3 + 2*x*y*z^3)*(x - y)*(y - z)/((3*x + 2*z)*(3*x + z)*(2*x + z)*(x + 2*z)*(x + z)*x*z))*q^-6 + O(q^-5)

The reduced series is the one which equals with the PT vertex (to be seen later).

#### Descendents

More generally, the vertex *with descendants* (for us) is

$$ V(\lambda, \mu, \nu; q; f) = \sum_{\pi \in \Pi(\lambda,\mu,\nu)} \frac{(-q)^{|\pi|} f(\pi)}{e(T_\pi \mathrm{Hilb}(\mathbb{C}^3))} \in H_{\mathsf{T}}^*(\mathrm{pt})_{\mathrm{loc}}((q)). $$

where $f$ is some function of the character of $\pi$. Usually "descendants" means that $f$ is a Chern character $\mathrm{ch}_k$, and $f(\pi)$ means $\mathrm{ch}_k((1-x)(1-y)(1-z)\chi_\pi)$, but in code $f$ can in principle be any function.

The Chern character $\mathrm{ch}_k(\mathcal{F})$ is implemented as `Cohomology.chern_character(k, F)`. Both `V.series_unreduced` and `V.series` allow descendants, using the parameter `descendant=f` where `f` must be a (Sage or Python) function.

Here is a single descendant $\mathrm{ch}_5$ in the Calabi-Yau limit.

In [21]:
V100 = BareVertex([1], [], [], Cohomology)
ch5 = lambda F: Cohomology.chern_character(5, F)
pretty( V100.series(3, x, y, -x-y, q, descendant=ch5) )

1/24*(x^2 + x*y + y^2)*(x + y)*x*y + (1/24*(17*x^2 + 29*x*y + 29*y^2)*(x + y)*x*y)*q + (1/24*(101*x^2 + 185*x*y + 185*y^2)*(x + y)*x*y)*q^2 + O(q^3)

**Exercise**: explain why $\mathrm{ch}_k((1-x)(1-y)(1-z)\chi_\pi) = 0$ for $k < 2$, for any vertex. Then explain it geometrically using Riemann-Roch. This is why people like to write $\mathrm{ch}_{k+2}$ in formulas instead of $\mathrm{ch}_k$.

In [22]:
ch1 = lambda F: Cohomology.chern_character(1, F)
BareVertex([3], [1,1], [2], Cohomology).series(3, x, y, z, q, descendant=ch1)

O(q^-5)

### DT vertex in K-theory

There is also a K-theoretic vertex, using the "K-theoretic Euler class" $\hat\wedge_{-1}^\bullet(-)^\vee$ instead of $e(-)$:

$$ V(\lambda, \mu, \nu; q; f) = \sum_{\pi \in \Pi(\lambda,\mu,\nu)} \frac{(-q)^{|\pi|} f(\pi)}{\hat\wedge_{-1}^\bullet(T_\pi \mathrm{Hilb}(\mathbb{C}^3))^\vee} \in K_{\mathsf{T}}^*(\mathrm{pt})_{\mathrm{loc}}((q)). $$

In [23]:
V000 = BareVertex([], [], [], KTheory) 
V000

Bare vertex with legs ([], [], []) using K-theory

#### The Nekrasov-Okounkov symmetrization $\hat\wedge_{-1}^\bullet$

The hat on $\hat\wedge_{-1}^\bullet(-)^\vee$ means to use

$$ \hat\wedge_{-1}^\bullet(\mathcal{L})^\vee = \wedge_{-1}^\bullet(\mathcal{L})^\vee \otimes \mathcal{L}^{1/2} = \mathcal{L}^{1/2} - \mathcal{L}^{-1/2} $$

instead of $\wedge_{-1}^\bullet(\mathcal{L})^\vee = 1 - \mathcal{L}^\vee$. One can opt out of using this symmetrization at the cost of no more nice closed-form formulas and DT/PT correspondence.

Due to the possibility of square roots, our implementation `KTheory.measure(f)` of $\hat\wedge_{-1}^\bullet(\mathcal{L})^\vee$ actually just returns $\mathcal{L} - \mathcal{L}^\vee$ instead of $\mathcal{L}^{1/2} - \mathcal{L}^{-1/2}$. Effectively this is a substitution $(x,y,z) \mapsto (x^2, y^2, z^2)$

In [24]:
V000series = V000.series_unreduced(2, x, y, z, q)
pretty(V000series)

1 + (-(x*y + 1)*(x*y - 1)*(x*z + 1)*(x*z - 1)*(y*z + 1)*(y*z - 1)/((x + 1)*(x - 1)*x*(y + 1)*(y - 1)*y*(z + 1)*(z - 1)*z))*q + O(q^2)

In [25]:
-V000series.coefficients()[1] == KTheory.measure(x + y + z - x*y - x*z - y*z)

True

#### Calabi-Yau and index limits

Like in cohomology, the *Calabi-Yau limit* is $xyz = 1$. We often write $\kappa = xyz$. In this limit, the K-theoretic vertex becomes the topological vertex.

There is also a combinatorial notion of *refined* topological vertex, a $1$-parameter deformation of the topological vertex; see [Iqbal-Kozcaz-Vafa]. It also has a combinatorial formula in terms of skew Schur functions. 

It can be obtained from the fully-equivariant K-theoretic DT vertex by a so-called *index limit* [Nekrasov-Okounkov], implemented as `KTheory.index_limit(f, s, kappa)`. Here $f$ is the original equivariant function, of $x$, $y$, $z$, and $\kappa$ is the desired resulting refined variable. The variable $s = (s_1, s_2, s_3)$ is a "slope" and should be set to $(N, -N-1, 1)$ for $N \gg 0$ for the $z$-axis to be the preferred direction. (Our $\kappa$ is [NO]'s $\kappa^{1/2}$.)

So that we don't have to set up different rings, let's just take $\kappa = x$.

In [26]:
N = 100000
index_limit = lambda f: KTheory.index_limit(f, (N, -N-1, 1), x)
V000.series_unreduced(4, x, y, z, q).map_coefficients(index_limit)

1 + x*q + (2*x^2 + 1)*q^2 + ((3*x^4 + 2*x^2 + 1)/x)*q^3 + O(q^4)

This is, of course, a terrible way to compute the refined vertex. Just use the [IKV] formulas instead.

#### Extended example: an explicit plethystic formula for the K-theoretic $V(\emptyset,\emptyset,\emptyset; q)$

Let $S^\bullet(-)$ denote the plethystic exponential. ($S^\bullet$ is like the symmetric algebra in the same way $\wedge_{-1}^\bullet$ is like the exterior algebra.) Then

$$ V(\emptyset, \emptyset, \emptyset; q) = S^\bullet\left(\frac{-q}{(1 - q\kappa)(1 - q/\kappa)} \hat\wedge_{-1}^\bullet(T_\pi \mathrm{Hilb}(\mathbb{C}^3)\right) $$

where $\pi$ is a single box and $\kappa = xyz$. Explicitly $T_\pi = x + y + z - xy - yz - xz$. This formula is a $\kappa$-deformation of the MacMahon function.

In [27]:
V000.series_unreduced(4, x, y, 1/(x*y), q)

1 + q + 3*q^2 + 6*q^3 + O(q^4)

We implement the plethystic exponential as

$$ S^\bullet(f) = \exp\left(\sum_{k > 0} \frac{\tau_k(f)}{k}\right) $$

where $\tau_k$ are the *Adams operations* defined by $\tau_k(\mathcal{L}) = \mathcal{L}^k$ on line bundles $\mathcal{L}$ and extended as algebra homomorphisms.

In [28]:
def adams(k):
    adams_wR = Hom(wR, wR)([x^k, y^k, z^k])
    adams_qR = Hom(qR, qR)([q^k])
    return adams_qR * Hom(qR, qR)(adams_wR) # lift adams_wR to Hom(qR, qR)

def plethystic_exponential(f, prec):
    g = sum(adams(k)(f)/k for k in range(1, prec)).add_bigoh(prec)
    return sum(g^k / factorial(k) for k in range(prec))

Sage is very detailed and thorough when it comes to morphisms.

In [29]:
adams(4)

Composite map:
  From: Power Series Ring in q over Fraction Field of Multivariate Polynomial Ring in x, y, z over Rational Field
  To:   Power Series Ring in q over Fraction Field of Multivariate Polynomial Ring in x, y, z over Rational Field
  Defn:   Ring endomorphism of Power Series Ring in q over Fraction Field of Multivariate Polynomial Ring in x, y, z over Rational Field
          Defn: Induced from base ring by
                Ring endomorphism of Fraction Field of Multivariate Polynomial Ring in x, y, z over Rational Field
                  Defn: x |--> x^4
                        y |--> y^4
                        z |--> z^4
        then
          Ring endomorphism of Power Series Ring in q over Fraction Field of Multivariate Polynomial Ring in x, y, z over Rational Field
          Defn: q |--> q^4

**Exercise**: check that $S^\bullet(q/(1-q)^2)$ really is the MacMahon function, using that $S^\bullet(x+y) = S^\bullet(x) S^\bullet(y)$ is multiplicative.

In [30]:
plethystic_exponential(q/(1-q)^2, 10)

1 + q + 3*q^2 + 6*q^3 + 13*q^4 + 24*q^5 + 48*q^6 + 86*q^7 + 160*q^8 + 282*q^9 + O(q^10)

Now check the actual formula for the vertex $V(\emptyset,\emptyset,\emptyset; q)$.

In [31]:
kappa = x*y*z
V000series == plethystic_exponential(-q/(1-q*kappa)/(1-q/kappa) * KTheory.measure(x+y+z - x*y-x*z-y*z), 3)

True

### PT vertices

All features above are implemented also for the Pandharipande--Thomas vertex `BareVertexPT`, with the same syntax. It is *much* faster to use `BareVertexPT.series` than `BareVertex.series`. The two are equal by the (conjectural) DT/PT correspondence, e.g.:

In [32]:
BareVertex([2], [1], [1], KTheory).series(3) == BareVertexPT([2], [1], [1], KTheory).series(3)

True

This is *stronger* than the K-theoretic DT/PT correspondence for partition functions, which are specific convolutions of vertices.

**Remark**. To really emphasize how much more computationally-efficient PT theory is, let's time some big computations.

In [33]:
%time f=BareVertex([2], [1], [1], KTheory).series(4)

CPU times: user 3min 39s, sys: 262 ms, total: 3min 39s
Wall time: 3min 38s


In [34]:
%time f=BareVertexPT([2], [1], [1], KTheory).series(4)

CPU times: user 1.02 s, sys: 25 µs, total: 1.02 s
Wall time: 1.02 s


#### Combinatorics of PT configurations

In [35]:
load("pt_configuration.sage")

PT vertices are computed using the same combinatorial formula as for DT vertices, but 3d partitions are replaced by PT configurations of boxes. These are much more combinatorially involved, and contain positive-dimensional families when all three legs are non-trivial.

In [36]:
PTConfigurations([2], [3], [4,2]).random_element_with_num_boxes(25).plot(colors=('green', 'yellow', 'white'))

Graphics3d Object

In the $1$-legged case, they are in some sense "complementary" to the $1$-legged 3d partitions of DT theory.

In [37]:
PTConfigurations([], [2,2,1], []).random_element_with_num_boxes(25).plot(colors=('green', 'yellow', 'white'))

Graphics3d Object

### Edges 

In [38]:
load("edge.sage")

Edge contributions are the same in any flavor of DT theory. The command `Edge(a, b, Cohomology)` creates an edge with normal bundle $\mathcal{O}(a) \oplus \mathcal{O}(b)$. Similarly there is `Edge(a, b, KTheory)`.

In [39]:
En1n1 = Edge(-1, -1, Cohomology)
En1n1

(-1, -1) edge using cohomology

The boxes carried by the edge form an infinite cylinder of the form $\lambda \times \mathbb{Z}$ for a partition $\lambda$. The contribution of this configuration to localization is given by the command `E.term_q(lamb, x, y, z, q=q)`, where:

- $x$ is the coordinate along the edge;
- $y$ is the coordinate in the $\mathcal{O}(a)$ direction;
- $z$ is the coordinate in the $\mathcal{O}(b)$ direction;

and $q$ is the box-counting variable as usual, recording Euler characteristic.

In [40]:
pretty( En1n1.term_q(Partition([3,1]), x, y, z, q=q) )

(1/2*(x + 3*y - z)*(x + 2*y)/(y - z)^2)*q^8

### Example: K-theoretic partition function of the conifold $\mathrm{tot}(\mathcal{O}_{\mathbb{P}^1}(-1) \oplus \mathcal{O}_{\mathbb{P}^1}(-1))$.

Introduce a K&auml;hler variable to keep track of degree along the $\mathbb{P}^1$.

In [41]:
AR.<A> = PowerSeriesRing(qR)

The resulting partition function will live in $K_{\mathsf{T}}(\mathrm{pt})_{\text{loc}}((q))[[A]]$. We compute it up to `prec` terms of precision in both $q$ and $A$.

In [42]:
def local_curve(a, b, prec):
    qq, AA = q.add_bigoh(prec), A.add_bigoh(prec)
    V = lambda lamb, mu, nu, x, y, z, qprec: BareVertexPT(lamb, mu, nu, KTheory).series(prec, x, y, z, qq)

    return sum( V([], mu, [], x, y, z, prec) * 
                Edge(a, b, KTheory).term_q(mu, y, z, x, qq) *
                V([], mu.conjugate(), [], z/y^a, 1/y, x/y^b, prec) *
                AA^mu.size() for nmu in range(prec) for mu in Partitions(nmu) )

pretty(local_curve(-1, -1, 3))

1 + O(q^3) + (-q + (-(x^2*y^2*z^2 + 1)/(x*y*z))*q^2 + (-(x^2*y^2*z^2 + x*y*z + 1)*(x^2*y^2*z^2 - x*y*z + 1)/(x^2*y^2*z^2))*q^3 + O(q^4))*A + (((x^2*y^2*z^2 + 1)/(x*y*z))*q^3 + ((x^2*y^2*z^2 + 1)^2/(x^2*y^2*z^2))*q^4 + ((2*x^4*y^4*z^4 + x^2*y^2*z^2 + 2)*(x^2*y^2*z^2 + 1)/(x^3*y^3*z^3))*q^5 + O(q^6))*A^2 + O(A^3)

**Exercise**: why are all coefficients *polynomials* in $\kappa$? (Hint: properness and rigidity.)

This is some massive pole-cancellation, since each vertex individually has many poles in various combinations of $x$, $y$, $z$.

To really emphasize the importance of properness in the exercise above, the same partition but for $\mathcal{O}(-1) \oplus \mathcal{O}(0)$ is not just a function of $\kappa$ (and doesn't have polynomial coefficients anymore).

In [43]:
pretty(local_curve(-1, 0, 2))

1 + O(q^2) + ((x/((x + 1)*(x - 1)))*q + ((x^2*y*z^2 + 1)/((x + 1)*(x - 1)*(y + 1)*z))*q^2 + O(q^3))*A + O(A^2)

**Exercise/hint for previous exercise**: why will the series above never have a pole at $y=1$?

### Example: K-theoretic partition function of the local curve $\mathrm{tot}(\mathcal{O}_{\mathbb{P}^1}(-2) \oplus \mathcal{O}_{\mathbb{P}^1}(0))$.

This example was suggested by Yannik Schuler.

In [44]:
Z20 = local_curve(-2, 0, 4)
pretty(Z20)

1 + O(q^4) + ((-(y*z + 1)*(y*z - 1)*x/((x + 1)*(x - 1)*y*z))*q + (-(x^2*y^2*z^2 + 1)*(y*z + 1)*(y*z - 1)/((x + 1)*(x - 1)*y^2*z^2))*q^2 + (-(x^2*y^2*z^2 + x*y*z + 1)*(x^2*y^2*z^2 - x*y*z + 1)*(y*z + 1)*(y*z - 1)/((x + 1)*(x - 1)*x*y^3*z^3))*q^3 + (-(x^4*y^4*z^4 + 1)*(x^2*y^2*z^2 + 1)*(y*z + 1)*(y*z - 1)/((x + 1)*(x - 1)*x^2*y^4*z^4))*q^4 + O(q^5))*A + (((y*z + x)*(y*z - x)*(y*z + 1)*(y*z - 1)*x^2/((x^2 + 1)*(x + 1)^2*(x - 1)^2*y^2*z^2))*q^2 + ((x^2*y^2*z^2 + 1)*(y*z + 1)^2*(y*z - 1)^2*x/((x + 1)^2*(x - 1)^2*y^3*z^3))*q^3 + ((x^6*y^6*z^6 + 2*x^4*y^6*z^6 - 2*x^6*y^4*z^4 + x^4*y^4*z^4 + 2*x^2*y^4*z^4 - 2*x^4*y^2*z^2 - x^2*y^2*z^2 + 2*y^2*z^2 - 2*x^2 - 1)*(y*z + 1)*(y*z - 1)/((x^2 + 1)*(x + 1)^2*(x - 1)^2*y^4*z^4))*q^4 + ((2*x^4*y^4*z^4 + x^2*y^2*z^2 + 2)*(x^2*y^2*z^2 + 1)*(y*z + 1)^2*(y*z - 1)^2/((x + 1)^2*(x - 1)^2*x*y^5*z^5))*q^5 + O(q^6))*A^2 + (((x^2 + y*z)*(x^2 - y*z)*(y*z + x)*(y*z - x)*(y*z + 1)*(y*z - 1)*x^3/((x^2 + x + 1)*(x^2 - x + 1)*(x^2 + 1)*(x + 1)^3*(x - 1)^3*y^3*z^3))*q^3 + (-(x^2*y^2*z^2 + 1)*(y*z + x)*(y*z - x)*(y*z + 1)^2*(y*z - 1)^2*x^2/((x^2 + 1)*(x + 1)^3*(x - 1)^3*y^4*z^4))*q^4 + (-(2*x^4*y^6*z^6 - 2*x^6*y^4*z^4 + x^4*y^4*z^4 + 2*x^2*y^4*z^4 - 2*x^4*y^2*z^2 - x^2*y^2*z^2 + 2*y^2*z^2 - 2*x^2)*(y*z + 1)^2*(y*z - 1)^2*x/((x^2 + 1)*(x + 1)^3*(x - 1)^3*y^5*z^5))*q^5 + (-(x^10*y^8*z^8 + 3*x^8*y^8*z^8 - 3*x^10*y^6*z^6 + 3*x^6*y^8*z^8 - 6*x^8*y^6*z^6 + 3*x^4*y^8*z^8 + 3*x^10*y^4*z^4 - 4*x^6*y^6*z^6 - x^6*y^4*z^4 + x^2*y^6*z^6 + x^8*y^2*z^2 - x^4*y^4*z^4 - 4*x^4*y^2*z^2 + 3*y^4*z^4 + 3*x^6 - 6*x^2*y^2*z^2 + 3*x^4 - 3*y^2*z^2 + 3*x^2 + 1)*(x^2*y^2*z^2 + 1)*(y*z + 1)*(y*z - 1)/((x^2 + x + 1)*(x^2 - x + 1)*(x^2 + 1)*(x + 1)^3*(x - 1)^3*y^6*z^6))*q^6 + O(q^7))*A^3 + O(A^4)

Check this answer against a closed-form plethystic formula; see [Nekrasov-Okounkov].

We must modify the definition of `adams` so that it acts on the K&auml;hler variable $A$ as well.

In [45]:
def adams(k):
    adams_wR = Hom(wR, wR)([x^k, y^k, z^k])
    adams_qR = Hom(qR, qR)([q^k])
    adams_AR = Hom(AR, AR)([A^k])
    return adams_AR * Hom(AR, AR)(adams_qR * Hom(qR, qR)(adams_wR))

def plethystic_exponential(f, prec):
    g = sum(adams(k)(f)/k for k in range(1, prec)).add_bigoh(prec)
    return sum(g^k / factorial(k) for k in range(prec))

In [46]:
Z20 == plethystic_exponential(A * q.add_bigoh(5) * x^2 * (1 - y^2*z^2) / ((1 - x^2) * (q - x*y*z) * (1 - q*x*y*z)), 4)

True